In [21]:
import numpy as np
from numba import jit
from profile import profile


@jit(nopython=True)
def sum_arrs(arrs):
    base = np.zeros(arrs[0].shape)
    for arr in arrs:
        base += arr
    return base


w = 1000
h = 1000
c = 3
n = 20

arrs = tuple(np.random.rand(h, w, c) for i in range(n))
masks = tuple(np.random.rand(h, w) for i in range(n))

@jit
def blend_arrs_1(arrs, masks):
    mask_sum = sum_arrs(masks)

    base = np.zeros(arrs[0].shape)
    for arr, mask in zip(arrs, masks):
        base += arr * np.true_divide(mask, mask_sum)[:, :, np.newaxis]
    return base

@jit(nopython=True)
def blend_arrs_2(arrs, masks):
    mask_sum = sum_arrs(masks)

    base = np.zeros(arrs[0].shape)
    for arr, mask in zip(arrs, masks):
        base += arr.transpose() * np.true_divide(mask, mask_sum).transpose()
    return base.transpose()

@jit(nopython=True)
def _blend_arrs_3(arrs, masks):
    mask_sum = sum_arrs(masks)

    base = np.zeros(arrs[0].shape)
    for arr, mask in zip(arrs, masks):
        base += arr * np.true_divide(mask, mask_sum)
    return base

def blend_arrs_3(arrs, masks):
    return _blend_arrs_3(arrs, tuple(mask[:, :, np.newaxis] for mask in masks))

@jit(nopython=True)
def blend_arrs_4(arrs, masks):
    mask_sum = sum_arrs(masks)

    base = np.zeros(arrs[0].shape)
    for arr, mask in zip(arrs, masks):
        base += arr * np.true_divide(mask, mask_sum).reshape(mask.shape[0], mask.shape[1], 1)
    return base

In [22]:
with profile():
    blend_arrs_1(arrs, masks)
    
with profile():
    blend_arrs_2(arrs, masks)

with profile():
    blend_arrs_3(arrs, masks)

with profile():
    blend_arrs_4(arrs, masks)

1.31654691696 seconds
5.85723519325 seconds
4.49706101418 seconds
4.28708291054 seconds
